<a href="https://colab.research.google.com/github/erturkkadir/sifirdan/blob/master/Ilac.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install chembl_webresource_client             # ChEMBL database   https://www.ebi.ac.uk/chembl/ bioactivity

ChEMBL Database, 2 milyondan fazla bilesenin biolojik aktivite kayitlari 

https://www.ebi.ac.uk/chembl/

In [ ]:
import pandas as pd                                                                                    # 
from chembl_webresource_client.new_client import new_client                        # chEMBL

In [ ]:
# Target search for coronavirus
target = new_client.target                                                                            # hedef molekul
target_query = target.search('acetylcholinesterase')                                        # acetylcholinesterase corona virus 
targets = pd.DataFrame.from_dict(target_query)                                            # dataFrame e cevir
targets.head(5)                                                                                           # Verileri goster

In [ ]:
selected_target = targets.target_chembl_id[0]                               # insandaki corona icin
activity = new_client.activity                                                        # ne kadar aktif
res = activity.filter(target_chembl_id=selected_target).filter(standard_type="IC50") # IC50 activite parametreesi 
df = pd.DataFrame.from_dict(res)                                                # DataFrame e cevir
df = df[df.standard_value.notna()]                                               # standart_value sutunundaki bos degerleri ayikla
df = df[df.canonical_smiles.notna()]                                             # canonical_smiles daki bos degerleri ayikla
df_nr = df.drop_duplicates(['canonical_smiles'])                            # tekrarlanmis degerleri ayikla
df_nr
len(df.canonical_smiles.unique())                                                 # Kac kayit kaldi elimizde
df_nr

Select and retrieve bioactivity data for Human Acetylcholinesterase (first entry)
We will assign the fifth entry (which corresponds to the target protein, Human Acetylcholinesterase) to the *selected_target* variable

In [ ]:
selection = ['molecule_chembl_id','canonical_smiles','standard_value']        # Sadece bu 3 sutun ile ilgileniyoruz
df2 = df_nr[selection]                                                                            # yeni bir degere ata
df2.head(5)

In [ ]:
bioactivity_threshold = []
for i in df2.standard_value:
  if float(i) >= 10000:
    bioactivity_threshold.append("inactive")
  elif float(i) <= 1000:
    bioactivity_threshold.append("active")
  else:
    bioactivity_threshold.append("intermediate")
bioactivity_class = pd.Series(bioactivity_threshold, name='class')
df3 = pd.concat([df2, bioactivity_class], axis=1)
df3
     

In [ ]:
df3.to_csv('acetylcholinesterase_03_bioactivity_data_curated.csv', index=False)